In [4]:
# access to our dataset on S3 and create Pandas data frame from it
import pandas as pd
import boto3

s3 = boto3.resource('s3')
# 's3' is a key word. create connection to S3 using default config and all buckets within S3

for bucket in s3.buckets.all():
    print(bucket.name)

sagemaker-studio-qodrzfd67o9
sagemaker-us-east-1-583582170591


In [9]:
s3_client = boto3.client('s3')

response = s3_client.create_bucket(
    ACL='public-read',
    Bucket='dsci-632',
)

print(response)

{'ResponseMetadata': {'RequestId': 'CMZM03HB401KKDJT', 'HostId': 'qtdcFSTvtBxZ93Ryj4Yo/K4xTxHFRswxoh1Jlf9dmsLVsgdcRHZiE02RxdYedy55BLC/teZFXiY=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'qtdcFSTvtBxZ93Ryj4Yo/K4xTxHFRswxoh1Jlf9dmsLVsgdcRHZiE02RxdYedy55BLC/teZFXiY=', 'x-amz-request-id': 'CMZM03HB401KKDJT', 'date': 'Sun, 10 Jul 2022 03:45:18 GMT', 'location': '/dsci-632', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 0}, 'Location': '/dsci-632'}


In [10]:
for bucket in s3.buckets.all():
    print(bucket.name)

dsci-632
sagemaker-studio-qodrzfd67o9
sagemaker-us-east-1-583582170591


In [11]:
import logging
from botocore.exceptions import ClientError
import os
import sys
import threading

class ProgressPercentage(object):

    def __init__(self, filename):
        self._filename = filename
        self._size = float(os.path.getsize(filename))
        self._seen_so_far = 0
        self._lock = threading.Lock()

    def __call__(self, bytes_amount):
        # To simplify, assume this is hooked up to a single filename
        with self._lock:
            self._seen_so_far += bytes_amount
            percentage = (self._seen_so_far / self._size) * 100
            sys.stdout.write(
                "\r%s  %s / %s  (%.2f%%)" % (
                    self._filename, self._seen_so_far, self._size,
                    percentage))
            sys.stdout.flush()

def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name,
                                         Callback=ProgressPercentage(file_name))
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [12]:
upload_file("../data/processed/CA_1_sales_data.csv", "dsci-632", "CA_1_sales_data.csv")

../data/processed/CA_1_sales_data.csv  219987 / 219987.0  (100.00%)

True

In [13]:
s3 = boto3.client('s3')

obj = s3.get_object(Bucket="dsci-632", Key="CA_1_sales_data.csv")
# get object and file (key) from bucket

df = pd.read_csv(obj['Body']) # 'Body' is a keyword
print(df.head())

   Unnamed: 0  wm_yr_wk  wday  month  year event_name_1 event_type_1  \
0  2011-01-29     11101     1      1  2011         None         None   
1  2011-01-30     11101     2      1  2011         None         None   
2  2011-01-31     11101     3      1  2011         None         None   
3  2011-02-01     11101     4      2  2011         None         None   
4  2011-02-02     11101     5      2  2011         None         None   

  event_name_2 event_type_2      cpi  ...  gas_price  snap_ca  ca_walmart  \
0         None         None  221.187  ...      3.163        0       39.50   
1         None         None  221.187  ...      3.163        0       42.50   
2         None         None  221.187  ...      3.155        0       36.50   
3         None         None  221.898  ...      3.155        1       42.40   
4         None         None  221.898  ...      3.155        1       38.69   

   ca_hobbies  ca_household  ca_foods  store_sales  median_price  min_price  \
0       20.14          36

In [14]:
upload_file("../data/processed/CA_2_sales_data.csv", "dsci-632", "CA_2_sales_data.csv")
upload_file("../data/processed/CA_3_sales_data.csv", "dsci-632", "CA_3_sales_data.csv")
upload_file("../data/processed/CA_4_sales_data.csv", "dsci-632", "CA_4_sales_data.csv")
upload_file("../data/processed/TX_1_sales_data.csv", "dsci-632", "TX_1_sales_data.csv")
upload_file("../data/processed/TX_2_sales_data.csv", "dsci-632", "TX_2_sales_data.csv")
upload_file("../data/processed/TX_3_sales_data.csv", "dsci-632", "TX_3_sales_data.csv")
upload_file("../data/processed/WI_1_sales_data.csv", "dsci-632", "WI_1_sales_data.csv")
upload_file("../data/processed/WI_2_sales_data.csv", "dsci-632", "WI_2_sales_data.csv")
upload_file("../data/processed/WI_3_sales_data.csv", "dsci-632", "WI_3_sales_data.csv")


../data/processed/WI_3_sales_data.csv  218867 / 218867.0  (100.00%)

True